In [1]:
import os
import sys
import pandas as pd
import csv
import numpy as np
import statistics

In [2]:
# READ DATASETS AND SAVE CSV DATA INTO PANDAS DATAFRAMES CASTING TIMESTAMPS

In [3]:
df_batch_registry = pd.read_csv(r'dataset/batch_registry.csv', sep = ";", header = 0, index_col = False)
df_batch_registry

,batch_id,arepa_type
0,b1,a1
1,b2,a2
2,b3,a1
3,b4,a3
4,b5,a4
5,b6,a1
6,b7,a1
7,b8,a2


In [4]:
df_cooking_metrics = pd.read_csv(r'dataset/cooking_metrics.csv', sep = ";", header = 0, index_col = False)
df_cooking_metrics['timestamp'] = pd.to_datetime(df_cooking_metrics['timestamp'])
df_cooking_metrics

,timestamp,machine_id,batch_id,metric_1,metric_2
0,2020-11-01 01:23:34,m1,b1,"0,1613","0,7684"
1,2020-11-01 01:25:34,m1,b2,"0,8544","0,3943"
2,2020-11-01 01:30:00,m1,b3,"0,1282","0,8135"
3,2020-11-01 02:00:34,m1,b4,"0,5657","0,6409"
4,2020-11-01 01:23:34,m2,b5,"0,3747","0,8199"
5,2020-11-01 01:25:34,m2,b6,"0,0248","0,4744"
6,2020-11-01 01:30:00,m2,b7,"0,6475","0,4514"
7,2020-11-01 02:00:34,m2,b8,"0,1206","0,4907"


In [5]:
df_faulty_intervals = pd.read_csv(r'dataset/faulty_intervals.csv', sep = ";", header = 0, index_col = False)
df_faulty_intervals['start_time'] = pd.to_datetime(df_faulty_intervals['start_time'])
df_faulty_intervals['end_time'] = pd.to_datetime(df_faulty_intervals['end_time'])
df_faulty_intervals

,machine_id,start_time,end_time
0,m1,2020-11-01 01:21:34,2020-11-01 01:25:00
1,m1,2020-11-02 11:41:54,2020-11-02 12:40:00
2,m2,2020-11-01 01:21:34,2020-11-01 01:25:00
3,m2,2020-11-02 11:41:54,2020-11-02 12:40:00
4,m2,2020-11-02 23:21:34,2020-11-02 23:25:00


In [6]:
# JOIN METRICS AND BATCH REGISTRY

In [7]:
df_metrics_join_arepas = pd.merge(df_cooking_metrics,df_batch_registry,on=['batch_id'])
df_metrics_join_arepas

,timestamp,machine_id,batch_id,metric_1,metric_2,arepa_type
0,2020-11-01 01:23:34,m1,b1,"0,1613","0,7684",a1
1,2020-11-01 01:25:34,m1,b2,"0,8544","0,3943",a2
2,2020-11-01 01:30:00,m1,b3,"0,1282","0,8135",a1
3,2020-11-01 02:00:34,m1,b4,"0,5657","0,6409",a3
4,2020-11-01 01:23:34,m2,b5,"0,3747","0,8199",a4
5,2020-11-01 01:25:34,m2,b6,"0,0248","0,4744",a1
6,2020-11-01 01:30:00,m2,b7,"0,6475","0,4514",a1
7,2020-11-01 02:00:34,m2,b8,"0,1206","0,4907",a2


In [8]:
# SOME DATA PREPARATION IN ORDER TO GET FLOAT TYPE FOR METRICS

In [9]:
df_metrics_join_arepas['metric_1'] = df_metrics_join_arepas['metric_1'].str.replace(',', '.')
df_metrics_join_arepas['metric_2'] = df_metrics_join_arepas['metric_2'].str.replace(',', '.')
df_metrics_join_arepas['metric_1'] = df_metrics_join_arepas['metric_1'].astype(float)
df_metrics_join_arepas['metric_2'] = df_metrics_join_arepas['metric_2'].astype(float)
df_metrics_join_arepas

,timestamp,machine_id,batch_id,metric_1,metric_2,arepa_type
0,2020-11-01 01:23:34,m1,b1,0.1613,0.7684,a1
1,2020-11-01 01:25:34,m1,b2,0.8544,0.3943,a2
2,2020-11-01 01:30:00,m1,b3,0.1282,0.8135,a1
3,2020-11-01 02:00:34,m1,b4,0.5657,0.6409,a3
4,2020-11-01 01:23:34,m2,b5,0.3747,0.8199,a4
5,2020-11-01 01:25:34,m2,b6,0.0248,0.4744,a1
6,2020-11-01 01:30:00,m2,b7,0.6475,0.4514,a1
7,2020-11-01 02:00:34,m2,b8,0.1206,0.4907,a2


In [10]:
# FILTERING DATASETS FOR MACHINE AND AREPAS TYPE

In [11]:
df_metrics_join_arepas_m1 = df_metrics_join_arepas.loc[df_metrics_join_arepas['machine_id'].str.contains('m1')]
df_metrics_join_arepas_m1_a1 = df_metrics_join_arepas_m1.loc[df_metrics_join_arepas_m1['arepa_type'].str.contains('a1')]
df_metrics_join_arepas_m1_a1

,timestamp,machine_id,batch_id,metric_1,metric_2,arepa_type
0,2020-11-01 01:23:34,m1,b1,0.1613,0.7684,a1
2,2020-11-01 01:30:00,m1,b3,0.1282,0.8135,a1


In [12]:
df_faulty_intervals_m1 = df_faulty_intervals.loc[df_faulty_intervals['machine_id'].str.contains('m1')]
df_faulty_intervals_m1

,machine_id,start_time,end_time
0,m1,2020-11-01 01:21:34,2020-11-01 01:25:00
1,m1,2020-11-02 11:41:54,2020-11-02 12:40:00


In [13]:
# CLEAN DATASET FROM FAULTY DATA

In [14]:
for index_1, row_1 in df_metrics_join_arepas_m1_a1.iterrows():
    for index_2, row_2 in df_faulty_intervals_m1.iterrows():
        if(row_2['start_time']<row_1['timestamp']< row_2['end_time']):
            print(row_1['timestamp'])
            df_metrics_join_arepas_m1_a1 = df_metrics_join_arepas_m1_a1.drop(index_1)
            break
df_metrics_join_arepas_m1_a1

2020-11-01 01:23:34


,timestamp,machine_id,batch_id,metric_1,metric_2,arepa_type
2,2020-11-01 01:30:00,m1,b3,0.1282,0.8135,a1


In [15]:
# FUNCTION TO COMPUTE HOURLY AVERAGE METRICS IN THE SPECIFIED INPUT TIMESTAMP

In [19]:
def compute_average(start_time_input, end_time_input):
    start_time = pd.to_datetime(start_time_input)
    end_time = pd.to_datetime(end_time_input)
    list_metric1 = []
    list_metric2 = []
    for index, row in df_metrics_join_arepas_m1_a1.iterrows():
        if(start_time<row['timestamp']< end_time):
            list_metric1.append(row['metric_1'])
            list_metric2.append(row['metric_2'])
    if(len(list_metric1)>0):
        avg_metric1 = statistics.mean(list_metric1)
        df_metrics_join_arepas_m1_a1['Avg_metric1'] = avg_metric1
        print('Average for metric1: ' + str(avg_metric1))
    else:
        print('No valid data for metric 1 in the specified interval')
    if(len(list_metric2)>0):
        avg_metric2 = statistics.mean(list_metric2)
        df_metrics_join_arepas_m1_a1['Avg_metric2'] = avg_metric1
        print('Average for metric2: ' + str(avg_metric2))
    else:
        print('No valid data for metric 2 in the specified interval')
    df_metrics_join_arepas_m1_a1['start_time'] = start_time
    df_metrics_join_arepas_m1_a1['end_time'] = end_time
    df_metrics_join_arepas_m1_a1.to_csv(r'dataset/.csv')

In [20]:
compute_average('2020-11-01T01:25:00','2020-11-01T01:36:00')

Average for metric1: 0.1282
Average for metric2: 0.8135


In [21]:
df_metrics_join_arepas_m1_a1

,timestamp,machine_id,batch_id,metric_1,metric_2,arepa_type,Avg_metric1,Avg_metric2,start_time,end_time
2,2020-11-01 01:30:00,m1,b3,0.1282,0.8135,a1,0.1282,0.1282,2020-11-01 01:25:00,2020-11-01 01:36:00
